In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import save_img

import numpy as np

import matplotlib.pyplot as plt

# Silence TQDM progress bar
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [2]:
param_img_size = (224,224)
param_batch_size = 1
# The data folder must contain one folder for each class
path_data = "./data_fer" 

In [3]:
# TODO Check label_mode is categorical as param
train_original_ds = image_dataset_from_directory(
        path_data,
        shuffle=False,
        
        seed=13,
        image_size=param_img_size,
        batch_size=param_batch_size,
    )

Found 1000 files belonging to 10 classes.


In [4]:
import tensorflow as tf
import tensorflow_addons as tfa
import cv2

contrast_factor = 2. # contrast_factor must be in the interval (-inf, inf).
def selected_transf(img):
  #img = tf.image.adjust_contrast(img, contrast_factor)
  img = tf.image.adjust_hue(
    img, 0.5)
  return img
  #x = tf.image.adjust_contrast(x, 1)

#f = lambda x: tf.image.adjust_contrast(x)

train_ds = (train_original_ds.map(
  lambda x, y: (selected_transf(x), y)))
# train1_ds.map(tf.image.adjust_contrast, 2.)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
import pandas as pd
df_classes = pd.read_csv('class_to_id.csv')
dict_classes = dict(zip(df_classes.name, df_classes.id))
class_names = train_original_ds.class_names

# dict_classes['guillotine']

In [6]:
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow.keras as keras

model = MobileNetV3Large(
    input_shape= (224,224,3),
    #alpha=1.0,
    minimalistic=False,
    #include_top=True,
    weights="imagenet",
    #input_tensor=None,
    classes=1000,
    #pooling=None,
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
) 

In [7]:
## See the prediction
predictions = model.predict(train_ds)
# results = tf.keras.applications.mobilenet.decode_predictions(predictions)


1000/1000 [==============================] - 16s 16ms/step


In [8]:
# Path
paths_save = train_original_ds.file_paths

In [9]:
# Labels
labels_save = [class_names[labels.numpy()] for _, labels in train_ds.unbatch()]

In [10]:
# Predictions probabilities
pred_prob_save = np.zeros((len(paths_save)))
for i in range(len(paths_save)):
    id_label = dict_classes[labels_save[i]]
    pred_prob_save[i] = predictions[i][id_label]

In [11]:
np.save('data_pred/paths.npy', paths_save)
np.save('data_pred/labels.npy', labels_save)
np.save('data_pred/prediction_prob.npy', pred_prob_save)

In [12]:
from lime import lime_image
from tensorflow.keras.preprocessing import image
from skimage.segmentation import mark_boundaries
import tensorflow as tf

explainer = lime_image.LimeImageExplainer()


C:\Users\simpletree\.conda\envs\tensorflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [15]:
# New paths to save masked images, class folders need to be created
new_paths = [s.replace('data_fer','data_fer_masked') for s in paths_save]
# new_img_masked = np.empty((len(train_ds), 224,224,3), dtype=(np.int32))
counter = 0
size_img = param_img_size[0]*param_img_size[1]
l_mask_sizes = []
for img, label in train_ds:
    explanation = explainer.explain_instance(
    image.img_to_array(img[0]).astype('double'), 
    model.predict, 
    top_labels=10, 
    hide_color=0, 
    num_samples=500)

    id_label = dict_classes[class_names[label[0]]]

    if id_label in explanation.top_labels:
        temp, mask = explanation.get_image_and_mask(id_label, positive_only=True, num_features=5, hide_rest=False)
        new_img = mark_boundaries(temp, mask).astype(np.int32)

        # Get how big is the mask
        size_mask = np.count_nonzero(mask)/size_img
        
        # Add size of mask to list
        l_mask_sizes.append(size_mask)
        
        # Every image is 50176 in total

#         new_img_masked[counter]=new_img
#       Save every image
        tf.keras.utils.save_img(new_paths[counter], new_img)
        #new_img.save(new_paths[counter], new_img)

    counter+=1

a_mask_sizes = np.array(l_mask_sizes)
np.save('data_pred/mask_sizes.npy', a_mask_sizes)

1/1 [==============================] - 0s 160ms/step


1/1 [==============================] - 0s 160ms/step


1/1 [==============================] - 0s 160ms/step


1/1 [==============================] - 0s 161ms/step


1/1 [==============================] - 0s 150ms/step


1/1 [==============================] - 0s 165ms/step


1/1 [==============================] - 0s 151ms/step


KeyboardInterrupt: 

In [ ]:
path_data_masked = './data_fer_masked/'
train_ds_masked = image_dataset_from_directory(
        path_data_masked,
        shuffle=False,
        
        seed=13,
        image_size=param_img_size,
        batch_size=param_batch_size,
    )

In [ ]:
## See the prediction
predictions_masked = model.predict(train_ds_masked)
# results_masked = tf.keras.applications.mobilenet.decode_predictions(predictions_masked)


In [ ]:
# Path
paths_masked = train_ds_masked.file_paths
paths_masked_fixed = [w.replace('data_fer_masked','data_fer') for w in paths_masked]

mask_sizes = np.load('data_pred/mask_sizes.npy')

df_masked = pd.DataFrame({'path':paths_masked_fixed, 'mask_size':mask_sizes})

In [ ]:
# Load original
paths_save = np.load('data_pred/paths.npy')
labels_save = np.load('data_pred/labels.npy')
pred_prob_save = np.load('data_pred/prediction_prob.npy')
df_original = pd.DataFrame({'path':paths_save,'label':labels_save, 'pred_original':pred_prob_save})

In [ ]:
df_joined = pd.concat([df_original.set_index('path'),df_masked.set_index('path')], axis=1, join='inner').reset_index()
df_joined['pred_masked'] = 0

In [ ]:
for i in range(len(df_joined)):
    id_label = dict_classes[df_joined['label'].iloc[i]]
    df_joined['pred_masked'].iloc[i] = predictions_masked[i][id_label]

In [ ]:
df_joined['impact_score'] = (df_joined['pred_original']-df_joined['pred_masked'])/df_joined['mask_size']

In [ ]:
df_joined